In [9]:
####株価データ統合####
import numpy as np
import pandas as pd
import matplotlib.pyplot  as plt
import numpy.matlib
import itertools
from scipy import sparse
from pandas.tools.plotting import scatter_matrix
from numpy.random import *
import re
import os
import MeCab
import neologdn
import sys

#np.random.seed(98537)

In [10]:
##データの読み込み
#読み込むデータの一覧
f = open("C:/statistics/data/stock_price/st-jp/f-list.txt")
code_name = f.read()  # ファイル終端まで全て読んだデータを返す
code_name = str.split(code_name, "\n")[1:] 
code = [re.sub(".CSV", "", code_name[i]) for i in range(len(code_name))]

In [11]:
#証券コードごとにデータを読み込み
n = len(code_name)-1
input_path = "C:/statistics/data/stock_price/st-jp/dd/{code_name}"
temporal_data = [i for i in range(n)]
id_list = [i for i in range(n)]
code_list = [i for i in range(n)]
for i in range(n):
    temporal_data[i] = np.array(pd.read_csv(input_path.format(code_name=code_name[i]), header=None))
    code_list[i] = np.repeat(code[i], temporal_data[i].shape[0])
    id_list[i] = np.repeat(i, temporal_data[i].shape[0])

In [126]:
#新しいディレクトリを作成


In [128]:
##業界ごとに株価データを書き出し
#新しいディレクトリを作成
company_info = pd.read_csv("C:/statistics/data/stock_price/st-jp/company_info.csv")[["コード", "銘柄名", "業種"]]
industry = np.unique(company_info["業種"])
no = np.append(np.array(["01", "02", "03", "04", "05", "06","07", "08", "09"]),
               np.array(np.arange(10, industry.shape[0]+1), "U2"))
industry_no = no + (np.array(np.repeat("_", no.shape[0]), "U2") + industry)
company_info_df = pd.merge(company_info, pd.DataFrame({"業種": industry, "業種no": industry_no}), on="業種", how="left")
company_info = np.array(company_info_df)

new_dir_path = "c:/statistics/data/stock_price/individual/{new_dir}"
for j in range(industry_no.shape[0]):
    if os.path.exists(new_dir_path.format(new_dir=industry_no[j]))==True:
        next
    else:
        os.mkdir(new_dir_path.format(new_dir=industry_no[j]))

#企業ごとにに株価データを出力
n = len(temporal_data)
index = np.array([0, 4])   #日付と終値のみ取得
for i in range(n):
    temporal_data0 = temporal_data[i][:, index]
    temp = company_info[company_info[:, 0]==np.array(code_list[i][0], dtype="int"), ].reshape(-1)
    code0 = temp[0]; company0 = temp[1]; industry0 = temp[3] 
    company0 = company0.replace("/", "_")   #「/」を置き換える
    out_df = pd.DataFrame({"company_ticker": code0, "date": temporal_data0[:, 0], "stock_price": temporal_data0[:, 1]})
    out_path = "c:/statistics/data/stock_price/individual/{industry}/{code}_{company}.csv"
    out_df.to_csv(out_path.format(industry=industry0, code=code0, company=company0), index=False, encoding="utf-8")

In [174]:
##業界コードを出力
industry_vec = np.array(company_info_df["業種no"])
for j in range(industry_no.shape[0]):
    index = np.where(industry_vec==industry_no[j])[0]
    tmp = company_info_df.iloc[index, ]
    tmp.index = np.arange(index.shape[0])
    joint_df = pd.concat([pd.DataFrame({"industry_code": np.repeat(industry_no[j][0:2], index.shape[0]), 
                                        "industry_name": np.repeat(industry_no[j][3:], index.shape[0])}), 
                          tmp[["コード", "銘柄名"]]], axis=1)
    joint_df.columns = ["industry_code", "industry_name", "company_ticker", "company_name"]

    out_path = "c:/statistics/data/stock_price/individual/{industry_no}.csv"
    joint_df.to_csv(out_path.format(industry_no=industry_no[j]), index=False, encoding="utf-8")

In [4]:
##すべてのデータをパネルデータとして書き出す
#リストに変換
pannel_data = np.array(list(itertools.chain(*[temporal_data[i] for i in range(n)])))
code = np.array(list(itertools.chain(*[code_list[i] for i in range(n)])))
r_id = np.array(list(itertools.chain(*[id_list[i] for i in range(n)])))
del temporal_data, code_list, id_list

In [5]:
#データフレームに加工
pannel_df = pd.DataFrame(pannel_data)
pannel_df.columns = ["日付", "始値", "高値", "安値", "終値", "出来高", "信用売残", "信用買銭"]
reference_id = pd.DataFrame(r_id)
reference_id.columns = ["reference_id"]

In [6]:
#企業情報を結合
company_info = pd.read_csv("C:/statistics/data/stock_price/st-jp/company_info.csv")
company_info["銘柄名"] = company_info["銘柄名"].str.replace("建設業", "建設")
company_panel = company_info[["コード", "銘柄名", "業種"]].iloc[r_id, ]
company_panel.index = np.arange(company_panel.shape[0])
joint_data = pd.concat([reference_id, company_panel, pannel_df], axis=1)

In [7]:
#データの書き出し
out1 = "C:/statistics/data/stock_price/stock_panel_data.csv"
out2 = "C:/statistics/data/stock_price/st-jp/company_info.csv"
joint_data.to_csv(out1, sep=",")
company_info.to_csv(out2, sep=",")
joint_data

,reference_id,コード,銘柄名,業種,日付,始値,高値,安値,終値,出来高,信用売残,信用買銭
0,0,1301,極洋,水産・農林,2000/01/04,1350,1350,1330,1330,163000,194000,1693000
1,0,1301,極洋,水産・農林,2000/01/05,1370,1450,1350,1350,218000,194000,1693000
2,0,1301,極洋,水産・農林,2000/01/06,1360,1380,1350,1350,194000,194000,1693000
3,0,1301,極洋,水産・農林,2000/01/07,1380,1410,1360,1370,211000,241000,1568000
4,0,1301,極洋,水産・農林,2000/01/11,1370,1380,1350,1350,236000,241000,1568000
5,0,1301,極洋,水産・農林,2000/01/12,1360,1380,1350,1370,251000,241000,1568000
6,0,1301,極洋,水産・農林,2000/01/13,1380,1430,1370,1390,183000,241000,1568000
7,0,1301,極洋,水産・農林,2000/01/14,1400,1450,1380,1410,493000,202000,1467000
8,0,1301,極洋,水産・農林,2000/01/17,1450,1630,1440,1600,391000,202000,1467000
9,0,1301,極洋,水産・農林,2000/01/18,1570,1570,1470,1520,195000,202000,1467000


,reference_id,コード,銘柄名,業種,日付,始値,高値,安値,終値,出来高,信用売残,信用買銭
0,0,1301,極洋,水産・農林,2000/01/04,1350,1350,1330,1330,163000,194000,1693000
1,0,1301,極洋,水産・農林,2000/01/05,1370,1450,1350,1350,218000,194000,1693000
2,0,1301,極洋,水産・農林,2000/01/06,1360,1380,1350,1350,194000,194000,1693000
3,0,1301,極洋,水産・農林,2000/01/07,1380,1410,1360,1370,211000,241000,1568000
4,0,1301,極洋,水産・農林,2000/01/11,1370,1380,1350,1350,236000,241000,1568000
5,0,1301,極洋,水産・農林,2000/01/12,1360,1380,1350,1370,251000,241000,1568000
6,0,1301,極洋,水産・農林,2000/01/13,1380,1430,1370,1390,183000,241000,1568000
7,0,1301,極洋,水産・農林,2000/01/14,1400,1450,1380,1410,493000,202000,1467000
8,0,1301,極洋,水産・農林,2000/01/17,1450,1630,1440,1600,391000,202000,1467000
9,0,1301,極洋,水産・農林,2000/01/18,1570,1570,1470,1520,195000,202000,1467000
